In [44]:
# Install Whisper, OpenSMILE dependencies, etc.
!pip install git+https://github.com/openai/whisper.git
!pip install opensmile
!pip install pydub
!pip install nltk
!pip install scikit-learn
!apt-get install sox ffmpeg  # needed for audio tools

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-t8w939tu
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-t8w939tu
  Resolved https://github.com/openai/whisper.git to commit c0d2f624c09dc18e709e37c2ad90c039a4eb72a2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
sox is already the newest version (14.4.2+git20190427-2+deb11u2ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [45]:
import whisper
import os
import json
import opensmile
import numpy as np
import pandas as pd
import nltk
from pydub import AudioSegment
from sklearn.cluster import KMeans
from collections import defaultdict
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [46]:
from google.colab import drive
drive.mount('/content/drive')

# Define the path to your audio file
audio_path = '/content/drive/MyDrive/Audios/P48.wav'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Transcribe & Segment with Whisper



In [47]:
import whisper

# Load Whisper model (medium size is a good trade-off)
model = whisper.load_model("medium")

# Transcribe the audio
result = model.transcribe(audio_path, verbose=True)

# Extract segments
segments = result['segments']

# Display a few segments
for s in segments[:3]:
    print(f"[{s['start']:.2f} - {s['end']:.2f}] {s['text']}")

/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Detecting language using up to the first 30 seconds. Use `--language` to specify the language
Detected language: English
[00:00.000 --> 00:02.000]  How are you? I'm doing well.
[00:02.000 --> 00:05.000]  Excellent. So, please tell me about yourself.
[00:05.000 --> 00:09.000]  I'm a student. I study computer science.
[00:09.000 --> 00:16.000]  I'm in my last year, so when I graduate in May, I'm going to be working on computer science stuff.
[00:16.000 --> 00:20.000]  I'm going to ask you a series of questions.
[00:20.000 --> 00:25.000]  To start with, tell me at a time when you demonstrated leadership.
[00:25.000 --> 00:31.000]  I demonstrated leadership on the track team. I was senior at Walter and the team.
[00:31.000 --> 00:37.000]  So, right now we're doing the conditioning and stuff like that all throughout the preseason.
[00:37.000 --> 00:40.000]  A lot of professional come and it's important to get them up to speed.
[00:40.000 --> 00:46.000]  So the seniors on the team, myself in

# Slice Audio into Sentence-Level Segments

In [48]:
from pydub import AudioSegment
import os

# Load full audio
full_audio = AudioSegment.from_wav(audio_path)

# Create a directory to store segments
os.makedirs("segments", exist_ok=True)

# Save each sentence segment to a separate file
segment_paths = []
for i, seg in enumerate(segments):
    start_ms = int(seg['start'] * 1000)
    end_ms = int(seg['end'] * 1000)

    if end_ms - start_ms < 300:
        print(f"⚠️ Skipping segment {i} — too short.")
        continue

    segment_audio = full_audio[start_ms:end_ms]
    seg_path = f"segments/segment_{i:03d}.wav"
    segment_audio.export(seg_path, format="wav")
    segment_paths.append({
        "index": i,
        "start": seg['start'],
        "end": seg['end'],
        "text": seg['text'],
        "path": seg_path
    })

print(f"✅ Saved {len(segment_paths)} valid segments.")


✅ Saved 50 valid segments.


# Extract eGeMAPSv02 Features using OpenSMILE

In [49]:
import opensmile

# Initialize OpenSMILE with eGeMAPSv02
smile = opensmile.Smile(
    feature_set=opensmile.FeatureSet.eGeMAPSv02,
    feature_level=opensmile.FeatureLevel.Functionals
)

# Extract features for all segments
feature_data = []
for seg in segment_paths:
    features = smile.process_file(seg["path"])
    features["index"] = seg["index"]
    feature_data.append(features)

# Combine all into a single DataFrame
features_df = pd.concat(feature_data).reset_index(drop=True)

# Speech Rate Analysis using KMeans

In [50]:
from sklearn.cluster import KMeans

# Calculate speech rate: words per second
for seg in segment_paths:
    num_words = len(seg["text"].split())
    duration = seg["end"] - seg["start"]
    seg["word_count"] = num_words
    seg["duration"] = duration
    seg["speech_rate"] = num_words / duration if duration > 0 else 0


In [51]:
# Extract speech rates
speech_rates = np.array([seg["speech_rate"] for seg in segment_paths]).reshape(-1, 1)

# Run KMeans clustering (3 clusters: slow, medium, fast)
kmeans_speech = KMeans(n_clusters=3, random_state=0, n_init='auto')
clusters = kmeans_speech.fit_predict(speech_rates)

# Attach cluster label to each segment
for seg, label in zip(segment_paths, clusters):
    seg["speech_rate_cluster"] = int(label)

In [52]:
# Create mapping to normalize cluster labels: 0 = slowest, 2 = fastest
cluster_avg_rates = {}
for label in range(3):
    avg_rate = np.mean([seg["speech_rate"] for seg in segment_paths if seg["speech_rate_cluster"] == label])
    cluster_avg_rates[label] = avg_rate

# Sort by avg rate and remap labels
sorted_clusters = sorted(cluster_avg_rates.items(), key=lambda x: x[1])
cluster_remap = {old: new for new, (old, _) in enumerate(sorted_clusters)}

# Apply remapped labels
for seg in segment_paths:
    seg["speech_rate_label"] = cluster_remap[seg["speech_rate_cluster"]]

In [53]:
for seg in segment_paths[:5]:
    print(f"[{seg['start']:.2f}–{seg['end']:.2f}] '{seg['text']}' | Rate: {seg['speech_rate']:.2f} w/s | Cluster: {seg['speech_rate_label']}")

[0.00–2.00] ' How are you? I'm doing well.' | Rate: 3.00 w/s | Cluster: 1
[2.00–5.00] ' Excellent. So, please tell me about yourself.' | Rate: 2.33 w/s | Cluster: 1
[5.00–9.00] ' I'm a student. I study computer science.' | Rate: 1.75 w/s | Cluster: 0
[9.00–16.00] ' I'm in my last year, so when I graduate in May, I'm going to be working on computer science stuff.' | Rate: 2.86 w/s | Cluster: 1
[16.00–20.00] ' I'm going to ask you a series of questions.' | Rate: 2.25 w/s | Cluster: 1


In [54]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans

# === Step 1: Compute speech rate for each segment ===
for seg in segment_paths:
    num_words = len(seg["text"].split())
    duration = seg["end"] - seg["start"]
    seg["word_count"] = num_words
    seg["duration"] = duration
    seg["speech_rate"] = num_words / duration if duration > 0 else 0

# === Step 2: KMeans clustering ===
speech_rates = np.array([seg["speech_rate"] for seg in segment_paths]).reshape(-1, 1)
kmeans = KMeans(n_clusters=3, random_state=0, n_init="auto")
clusters = kmeans.fit_predict(speech_rates)

# === Step 3: Map clusters to speech rate labels ===
for seg, label in zip(segment_paths, clusters):
    seg["rate_cluster"] = int(label)

# Normalize cluster IDs so: 0 = Slowest, 2 = Fastest
cluster_avg = {
    label: np.mean([seg["speech_rate"] for seg in segment_paths if seg["rate_cluster"] == label])
    for label in range(3)
}
sorted_labels = sorted(cluster_avg.items(), key=lambda x: x[1])
remap = {old: new for new, (old, _) in enumerate(sorted_labels)}

# Apply remapped labels
for seg in segment_paths:
    seg["normalized_cluster"] = remap[seg["rate_cluster"]]

# Define human-readable speech rate labels
speech_rate_labels = {
    0: "Slow",
    1: "Moderate",
    2: "Fast"
}

# === Step 4: Build DataFrame and Print Output ===
df_speech_rate = pd.DataFrame([
    {
        "timestamp": f"{seg['start']:.2f} – {seg['end']:.2f}",
        "cluster": seg["normalized_cluster"],
        "label": speech_rate_labels[seg["normalized_cluster"]]
    }
    for i, seg in enumerate(segment_paths)
])

print(df_speech_rate)


          timestamp  cluster     label
0       0.00 – 2.00        1  Moderate
1       2.00 – 5.00        1  Moderate
2       5.00 – 9.00        0      Slow
3      9.00 – 16.00        1  Moderate
4     16.00 – 20.00        1  Moderate
5     20.00 – 25.00        1  Moderate
6     25.00 – 31.00        1  Moderate
7     31.00 – 37.00        1  Moderate
8     37.00 – 40.00        2      Fast
9     40.00 – 46.00        1  Moderate
10    46.00 – 53.00        1  Moderate
11    53.00 – 61.00        1  Moderate
12    61.00 – 64.00        1  Moderate
13    64.00 – 77.00        0      Slow
14    77.00 – 80.00        2      Fast
15    80.00 – 84.00        2      Fast
16    84.00 – 88.00        1  Moderate
17    88.00 – 93.00        2      Fast
18    93.00 – 99.00        0      Slow
19   99.00 – 105.00        0      Slow
20  105.00 – 111.00        0      Slow
21  111.00 – 115.00        2      Fast
22  115.00 – 119.00        2      Fast
23  119.00 – 123.00        0      Slow
24  123.00 – 134.00      

#  Train a Random Forest model to predict clarity-related scores

Train Random Forest Model

R² (Coefficient of Determination):

Ranges from −∞ to 1

A value of 1.0 means perfect prediction

0 means the model does no better than predicting the mean

Negative values indicate the model is doing worse than just guessing the average.

MAE (Mean Absolute Error):

Measures average prediction error in the same scale as the labels (usually [0–1] or Likert-type scores)

Lower is better; closer to 0 is ideal

Only EngagingTone shows moderate predictive success (R² ≈ 0.51).

SpeakingRate performs reasonably well due to its strong acoustic correlation with features like MFCCs and loudness.

StructuredAnswers, Focused, and NotAwkward are weak—these likely depend on semantic or content-based cues, which OpenSMILE features alone can't capture.

Negative R² for Focused suggests it might not be learnable from current features or labels are inconsistent.

Updated: Keeping Focused, Authentic, NotAwkward, EngagingTone

In [55]:
import pandas as pd
import numpy as np
import glob
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import cross_val_score, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer, mean_absolute_error, r2_score

# === Step 1: Load feature files ===
features_folder = "/content/drive/MyDrive/Audios/Segmented Interview Information"
feature_files = glob.glob(features_folder + "/*_all_features.csv")

# Define OpenSMILE acoustic features
clarity_features = [
    "F0semitoneFrom27.5Hz_sma3nz_amean",
    "HNRdBACF_sma3nz_amean",
    "jitterLocal_sma3nz_amean",
    "shimmerLocaldB_sma3nz_amean",
    "loudness_sma3_amean",
    "mfcc1_sma3_amean", "mfcc2_sma3_amean", "mfcc3_sma3_amean"
]

# === Step 2: Aggregate features for each participant ===
agg_data = []
for file in feature_files:
    df = pd.read_csv(file)
    participant = file.split("/")[-1].split("_")[0].lower()

    if not set(clarity_features).issubset(df.columns):
        print(f"⚠️ Skipping {participant} due to missing features")
        continue

    subset = df[clarity_features].dropna()
    if subset.empty:
        print(f"⚠️ Skipping {participant} — no valid rows.")
        continue

    agg_mean = subset.mean()
    agg_std = subset.std()
    agg_min = subset.min()
    agg_max = subset.max()

    agg_vector = pd.concat([agg_mean, agg_std, agg_min, agg_max])
    agg_vector.index = [f"{stat}_{col}" for stat in ["mean", "std", "min", "max"] for col in clarity_features]
    agg_vector["Participant"] = participant

    agg_data.append(agg_vector)

agg_features_df = pd.DataFrame(agg_data)

# === Step 3: Load scores and keep AGGR ===
score_path = "/content/drive/MyDrive/Audios/turker_scores_full_interview.csv"
scores_df = pd.read_csv(score_path)
scores_df["Participant"] = scores_df["Participant"].str.lower()
scores_df = scores_df[scores_df["Worker"] == "AGGR"]

# Keep only relevant clarity labels
clarity_labels = ["Focused", "Authentic", "NotAwkward", "EngagingTone"]
scores_df = scores_df[["Participant"] + clarity_labels]

# === Step 4: Merge features and scores ===
merged = pd.merge(agg_features_df, scores_df, on="Participant")
X = merged.drop(columns=["Participant"] + clarity_labels)
y = merged[clarity_labels]

# === Step 5: Train model ===
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

rf = RandomForestRegressor(n_estimators=100, random_state=42)
multi_rf = MultiOutputRegressor(rf)

kf = KFold(n_splits=5, shuffle=True, random_state=42)
r2_scores, mae_scores = [], []

for train_idx, test_idx in kf.split(X_scaled):
    X_train, X_test = X_scaled[train_idx], X_scaled[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    multi_rf.fit(X_train, y_train)
    y_pred = multi_rf.predict(X_test)

    r2_scores.append(r2_score(y_test, y_pred, multioutput='raw_values'))
    mae_scores.append(mean_absolute_error(y_test, y_pred, multioutput='raw_values'))

# === Step 6: Report Results ===
r2_scores = np.array(r2_scores)
mae_scores = np.array(mae_scores)

print("✅ Multi-Output Random Forest Clarity Prediction (Updated - 4 Labels)")
for i, label in enumerate(clarity_labels):
    print(f"\n🎯 {label}")
    print(f"  Mean R²:  {np.mean(r2_scores[:, i]):.3f}")
    print(f"  Mean MAE: {np.mean(mae_scores[:, i]):.3f}")


✅ Multi-Output Random Forest Clarity Prediction (Updated - 4 Labels)

🎯 Focused
  Mean R²:  -0.111
  Mean MAE: 0.386

🎯 Authentic
  Mean R²:  0.140
  Mean MAE: 0.293

🎯 NotAwkward
  Mean R²:  0.100
  Mean MAE: 0.577

🎯 EngagingTone
  Mean R²:  0.511
  Mean MAE: 0.488


Testing out with Sample Audio

In [56]:
import os
import numpy as np
import pandas as pd
import opensmile
from pydub import AudioSegment
from IPython.display import display

# === Load audio ===
audio_path = "/content/drive/MyDrive/Audios/P48.wav"
full_audio = AudioSegment.from_wav(audio_path)
duration_ms = len(full_audio)

# === Parameters ===
window_ms = 5000     # 5 seconds
stride_ms = 2000     # 2 seconds
results = []

# === OpenSMILE extractor ===
smile = opensmile.Smile(
    feature_set=opensmile.FeatureSet.eGeMAPSv02,
    feature_level=opensmile.FeatureLevel.Functionals
)

# === Timestamp formatter ===
def format_timestamp(seconds):
    mins = int(seconds) // 60
    secs = int(seconds) % 60
    return f"{mins:02d}:{secs:02d}"

# === Label mapping function ===
def label_by_dimension(label, score):
    if label == "Focused":
        if score >= 4.5: return "Highly Focused"
        elif score >= 3.5: return "Moderately Focused"
        elif score >= 2.5: return "Somewhat Distracted"
        else: return "Unfocused"
    elif label == "Authentic":
        if score >= 4.5: return "Genuine"
        elif score >= 3.5: return "Sincere"
        elif score >= 2.5: return "Neutral"
        else: return "Inauthentic"
    elif label == "NotAwkward":
        if score >= 4.5: return "Natural"
        elif score >= 3.5: return "Comfortable"
        elif score >= 2.5: return "Mildly Awkward"
        else: return "Awkward"
    elif label == "EngagingTone":
        if score >= 4.5: return "Very Engaging"
        elif score >= 3.5: return "Engaging"
        elif score >= 2.5: return "Somewhat Flat"
        else: return "Monotone"
    else:
        return "Unknown"

# === Segment and predict ===
segment_index = 0
for start_ms in range(0, duration_ms - window_ms + 1, stride_ms):
    end_ms = start_ms + window_ms
    segment_audio = full_audio[start_ms:end_ms]
    seg_path = "temp_segment.wav"
    segment_audio.export(seg_path, format="wav")

    try:
        features = smile.process_file(seg_path).reset_index(drop=True)
        if features.empty:
            continue

        # Aggregate statistics
        mean_vec = features.mean()
        std_vec = features.std()
        min_vec = features.min()
        max_vec = features.max()

        combined = pd.concat([mean_vec, std_vec, min_vec, max_vec])
        combined.index = [f"{stat}_{col}" for stat in ["mean", "std", "min", "max"] for col in features.columns]

        # Build RF input
        full_vector = pd.DataFrame(columns=X.columns)
        for col in full_vector.columns:
            if col in combined.index:
                full_vector.at[0, col] = combined[col]
        full_vector = full_vector.fillna(0.0)

        # Scale and predict
        scaled = scaler.transform(full_vector)
        pred_vector = multi_rf.predict(scaled)[0]
        clarity_scores = dict(zip(clarity_labels, np.round(pred_vector, 2)))

        # Add labels
        labeled_scores = {f"{k}_label": label_by_dimension(k, v) for k, v in clarity_scores.items()}

        # Append full row
        results.append({
            "index": segment_index,
            "timestamp": f"{format_timestamp(start_ms / 1000)} - {format_timestamp(end_ms / 1000)}",
            **clarity_scores,
            **labeled_scores
        })

        segment_index += 1

    except Exception as e:
        print(f"⚠️ Segment {segment_index} error: {e}")
        continue

# === Display as table ===
df = pd.DataFrame(results)
display(df[["index", "timestamp"] +
          [f"{k}" for k in clarity_scores.keys()] +
          [f"{k}_label" for k in clarity_scores.keys()]])


/tmp/ipython-input-56-1859742739.py:82: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  full_vector = full_vector.fillna(0.0)
/tmp/ipython-input-56-1859742739.py:82: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  full_vector = full_vector.fillna(0.0)
/tmp/ipython-input-56-1859742739.py:82: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  full_v

,index,timestamp,Focused,Authentic,NotAwkward,EngagingTone,Focused_label,Authentic_label,NotAwkward_label,EngagingTone_label
0,0,00:00 - 00:05,5.50,5.43,5.26,4.48,Highly Focused,Genuine,Natural,Engaging
1,1,00:02 - 00:07,5.46,5.40,5.16,4.13,Highly Focused,Genuine,Natural,Engaging
2,2,00:04 - 00:09,5.48,5.37,5.41,4.15,Highly Focused,Genuine,Natural,Engaging
3,3,00:06 - 00:11,5.43,5.39,5.24,3.74,Highly Focused,Genuine,Natural,Engaging
4,4,00:08 - 00:13,5.44,5.40,5.17,3.80,Highly Focused,Genuine,Natural,Engaging
...,...,...,...,...,...,...,...,...,...,...
141,141,04:42 - 04:47,5.39,5.40,5.26,3.83,Highly Focused,Genuine,Natural,Engaging
142,142,04:44 - 04:49,5.44,5.41,5.32,3.94,Highly Focused,Genuine,Natural,Engaging
143,143,04:46 - 04:51,5.53,5.41,5.24,4.21,Highly Focused,Genuine,Natural,Engaging
144,144,04:48 - 04:53,5.43,5.39,4.70,3.37,Highly Focused,Genuine,Natural,Somewhat Flat


Improved version of detecting long pauses within sentence\

In [57]:
import numpy as np
from pydub import AudioSegment

# === Load mono audio ===
audio_path = "/content/drive/MyDrive/Audios/P48.wav"
audio = AudioSegment.from_wav(audio_path).set_channels(1)
sample_rate = audio.frame_rate
duration_ms = len(audio)

# === Parameters ===
window_ms = 5000       # 5 seconds
stride_ms = 2000       # 2 seconds
frame_ms = 100         # frame size for energy calculation
frame_samples = int(sample_rate * (frame_ms / 1000))
energy_threshold = 0.02
pause_threshold_sec = 0.6

# === Sliding window analysis ===
print("📊 Long Pause Detection (Sliding Window 5s / 2s overlap):\n")
window_index = 0

for start_ms in range(0, duration_ms - window_ms + 1, stride_ms):
    end_ms = start_ms + window_ms
    segment_audio = audio[start_ms:end_ms]
    samples = np.array(segment_audio.get_array_of_samples())

    energies = []
    times = []

    for i in range(0, len(samples) - frame_samples + 1, frame_samples):
        frame = samples[i:i + frame_samples]
        energy = np.sum(frame.astype(np.float32) ** 2)
        norm_energy = energy / (np.max(samples.astype(np.float32) ** 2) + 1e-8)
        energies.append(norm_energy)
        times.append(i / sample_rate)

    # Detect low energy frames
    low_energy = np.array(energies) < energy_threshold
    pause_start = None
    long_pause_detected = False

    for idx, val in enumerate(low_energy):
        if val and pause_start is None:
            pause_start = times[idx]
        elif not val and pause_start is not None:
            pause_end = times[idx]
            if pause_end - pause_start >= pause_threshold_sec:
                long_pause_detected = True
                break  # Only need to detect one long pause
            pause_start = None

    # Output
    timestamp = f"{start_ms/1000:.2f}s - {end_ms/1000:.2f}s"
    status = "Yes" if long_pause_detected else "No"
    print(f"{timestamp} | {status}")

    window_index += 1


📊 Long Pause Detection (Sliding Window 5s / 2s overlap):

0.00s - 5.00s | No
2.00s - 7.00s | No
4.00s - 9.00s | No
6.00s - 11.00s | No
8.00s - 13.00s | No
10.00s - 15.00s | No
12.00s - 17.00s | No
14.00s - 19.00s | No
16.00s - 21.00s | No
18.00s - 23.00s | No
20.00s - 25.00s | No
22.00s - 27.00s | No
24.00s - 29.00s | No
26.00s - 31.00s | No
28.00s - 33.00s | No
30.00s - 35.00s | No
32.00s - 37.00s | No
34.00s - 39.00s | No
36.00s - 41.00s | No
38.00s - 43.00s | No
40.00s - 45.00s | No
42.00s - 47.00s | No
44.00s - 49.00s | No
46.00s - 51.00s | No
48.00s - 53.00s | No
50.00s - 55.00s | No
52.00s - 57.00s | No
54.00s - 59.00s | No
56.00s - 61.00s | No
58.00s - 63.00s | No
60.00s - 65.00s | No
62.00s - 67.00s | No
64.00s - 69.00s | No
66.00s - 71.00s | No
68.00s - 73.00s | No
70.00s - 75.00s | No
72.00s - 77.00s | No
74.00s - 79.00s | No
76.00s - 81.00s | No
78.00s - 83.00s | No
80.00s - 85.00s | No
82.00s - 87.00s | No
84.00s - 89.00s | No
86.00s - 91.00s | No
88.00s - 93.00s | No
90.00